In [2]:
import pandas as pd
import glob
import os
import spacy
import numpy as np
import sys
import re
from optparse import OptionParser
from nltk.stem.snowball import SnowballStemmer
from multiprocessing import cpu_count
from nltk.corpus import stopwords
from utils import sentence_processing, text_stemmer

ModuleNotFoundError: No module named 'spacy'

In [31]:
def read_txt(path):
    with open(path, 'r') as f:
        content = f.read()
    return content

In [20]:
# !{sys.executable} -m pip install pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz

  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz (15.6 MB)
     |████████████████████████████████| 5.8 MB 5.0 MB/s eta 0:00:01
  Using cached typer-0.3.2-py3-none-any.whl (21 kB)
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.4.0-py3-none-any.whl size=15660350 sha256=af8590a93f83e34b1f4d49ce7c5586f1a1403c82c4c3396eecfd1595ce89ed02
  Stored in directory: /home/destrada/.cache/pip/wheels/31/4b/3a/03f2732eb69789a7df3bf29c1bc010f660ada5f3c295d4073c
Successfully built en-core-sci-sm
  Attempting uninstall: click
    Found existing installation: click 8.0.3
    Uninstalling click-8.0.3:
      Successfully uninstalled click-8.0.3
  Attempting uninstall: typer
    Found existing installation: typer 0.4.0
    Uninstalling typer-0.4.0:
      Successfully uninstalled typer-0.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.1
    Uninstalli

In [32]:
disciplines = []
clinical_cases_v0_path = '/jupyter/Corpus_NLP4BIA/clinical_cases/V0/'
for discipline in os.listdir(clinical_cases_v0_path):
    disciplines.append(discipline)

In [33]:
nlp = spacy.load("en_core_sci_sm", disable=["ner", "lemmatizer"])
stop_words = stopwords.words("spanish")
pattern = re.compile(r'\b(' + r'|'.join(stop_words) + r')\b\s*')
snow_stemmer = SnowballStemmer(language='spanish')

/home/destrada/anaconda3/envs/tf/lib/python3.7/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_core_sci_sm' (0.4.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


### Calcular:
    + Número de documentos.
    + Número de frases.
    + Número de tokens.
    + Número de frases, tokens por documentos
    + Desviación típica (sd) 

In [36]:
num_sentences = []
tokens_length = []
path = 'articles_es.txt'
txt_wikipedia = read_txt(path)
file_sentences = list(nlp(txt_wikipedia).sents)
sentences_segmented = [sent.text.strip() for sent in file_sentences]
num_sentences.append(len(sentences_segmented))
for sent in sentences_segmented:
    #processed_sentence = sentence_processing(sent, pattern)
    tokenized_sentence = sent.split()
    tokens_length.append(len(tokenized_sentence))
                    
print(f'Num of sentences: {np.sum(num_sentences)}')
print(f'Num of tokens: {np.sum(tokens_length)}')

ValueError: [E088] Text of length 39111179 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.

In [22]:
analysis_dict = {}
for discipline in disciplines:
    discipline_path = os.path.join(clinical_cases_v0_path, discipline)
    if os.path.isdir(discipline_path):
        num_docs = 0
        num_sentences = []
        tokens_length = []
        for file_name in os.listdir(discipline_path):    
            if file_name.endswith('.txt'):
                num_docs += 1
                file_path = os.path.join(discipline_path, file_name)
                txt_file = open(file_path).read()
                file_sentences = list(nlp(txt_file).sents)
                sentences_segmented = [sent.text.strip() for sent in file_sentences]
                num_sentences.append(len(sentences_segmented))
                for sent in sentences_segmented:
                    processed_sentence = sentence_processing(sent, pattern)
                    tokenized_sentence = sent.split()
                    tokens_length.append(len(tokenized_sentence))

        analysis_dict[discipline] = {'#_docs': num_docs,
                                     '#_sentences': np.sum(num_sentences),
                                     '#_tokens': np.sum(tokens_length),
                                     'tokens/docs': np.sum(tokens_length)/num_docs,
                                     'sentences/docs': np.sum(num_sentences)/num_docs,
                                     'mean_token_length_per_sentence': round(np.mean(tokens_length), 3),
                                     'std_token_length_per_sentence': round(np.std(tokens_length), 3),
                                     'mean_sentences_per_doc': round(np.mean(num_sentences), 3),
                                     'std_sentences_per_doc': round(np.std(num_sentences), 3),
                                     }

In [23]:
analysis_df = pd.DataFrame.from_dict(analysis_dict, orient="index").reset_index(drop=False)

In [26]:
analysis_df.head()

,index,#_docs,#_sentences,#_tokens,tokens/docs,sentences/docs,mean_token_length_per_sentence,std_token_length_per_sentence,mean_sentences_per_doc,std_sentences_per_doc
0,geneccc,427,7323,144205,337.716628,17.149883,19.692,16.463,17.150,10.337
1,cardioccc,557,34960,501786,900.872531,62.764811,14.353,13.110,62.765,29.577
2,interccc,1020,17789,337148,330.537255,17.440196,18.953,13.512,17.440,11.001
3,covidccc,316,5969,122630,388.069620,18.889241,20.544,12.922,18.889,9.826
4,uroccc,440,11216,212930,483.931818,25.490909,18.984,13.689,25.491,12.618


In [29]:
print('Results:')
print(f"Total number of extracted sentences: {analysis_df['#_sentences'].sum()}")
print(f"Total number of extracted sentences: {analysis_df['#_tokens'].sum()}")

Results:
Total number of extracted sentences: 279179
Total number of extracted sentences: 5266807


In [129]:
analysis_df = analysis_df.drop(labels=20, axis=0)

In [130]:
analysis_df.round(3)

,index,#_docs,#_sentences,#_tokens,tokens/docs,sentences/docs,mean_token_length_per_sentence,std_token_length_per_sentence,mean_sentences_per_doc,std_sentences_per_doc
0,oftaccc,300,4263.0,90517.0,301.723,14.210,21.233,12.578,14.210,7.338
1,occuccc,101,2771.0,43057.0,426.307,27.436,15.538,12.971,27.436,37.998
2,endoccc,204,4467.0,95974.0,470.461,21.897,21.485,14.807,21.897,13.653
3,cardioccc,556,34969.0,501611.0,902.178,62.894,14.344,13.094,62.894,29.499
4,dermaccc,665,9768.0,199201.0,299.550,14.689,20.393,12.551,14.689,6.859
5,neumoccc,49,1117.0,20726.0,422.980,22.796,18.555,14.877,22.796,11.952
6,odonccc,45,874.0,20902.0,464.489,19.422,23.915,16.009,19.422,13.019
7,emeccc,202,4004.0,60565.0,299.827,19.822,15.126,13.441,19.822,11.574
8,reumaccc,384,7148.0,140838.0,366.766,18.615,19.703,17.167,18.615,17.671
9,spaccc,1000,16616.0,350770.0,350.770,16.616,21.110,14.486,16.616,7.853


In [131]:
analysis_df.rename({'index': 'medical_discipline'}, axis=1, inplace=True)

In [132]:
analysis_df.head()

,medical_discipline,#_docs,#_sentences,#_tokens,tokens/docs,sentences/docs,mean_token_length_per_sentence,std_token_length_per_sentence,mean_sentences_per_doc,std_sentences_per_doc
0,oftaccc,300,4263.0,90517.0,301.723333,14.210000,21.233,12.578,14.210,7.338
1,occuccc,101,2771.0,43057.0,426.306931,27.435644,15.538,12.971,27.436,37.998
2,endoccc,204,4467.0,95974.0,470.460784,21.897059,21.485,14.807,21.897,13.653
3,cardioccc,556,34969.0,501611.0,902.178058,62.893885,14.344,13.094,62.894,29.499
4,dermaccc,665,9768.0,199201.0,299.550376,14.688722,20.393,12.551,14.689,6.859


In [138]:
analysis_df.to_excel("text_statistical_analysis.xlsx", index=False)